In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
#from astropy.time import Time
import datetime as dt
import time
from IPython.display import clear_output
from matplotlib import gridspec

#from scipy.interpolate import interp1d
debug = False

In [ ]:
def get_latest_eve(debug=debug):
    # Add try to escape if error

    error = [0]

    eve_fn = "https://lasp.colorado.edu/eve/data_access/eve_data/quicklook/L0CS/LATEST_EVE_L0CS_DIODES_10s_counts.json"

    try:        
        df_eve = pd.read_json(eve_fn)
    except ValueError:
        error = [-10]
#        return [-1], [-1], [-1], [-1], [-1], [-1]
        return [error]
    
    #TaiOffset = 378691200 + 37
    
    #tt = Time(df_eve['TAI'] - TaiOffset, format='unix_tai', scale='tai')
    # can we do TAI conversion just in datetime?
    leap_seconds = 37
    ta0 = dt.datetime(1958,1,1,0,0,0,0) # TAI base time
    tt = []
    for ttime in df_eve['TAI']:
        tmp = ttime - leap_seconds
        ss = int(tmp)
        us = 1e6*(tmp-ss)
        tt.append(ta0 + dt.timedelta(seconds = ss, microseconds = us))
    
    #edate = np.asarray(tt.datetime)
    edate = np.asarray(tt)
    df_eve['dt'] = edate # add dates column
    df_eve['dt'] = pd.to_datetime(df_eve['dt']) # force it to datetime

    e0 = np.asarray(df_eve['ESP_0_7_COUNTS'])
    e30 = np.asarray(df_eve['ESP_30_COUNTS'])
    
    # Make averaged dataframe
    df_slow=df_eve.groupby(pd.Grouper(key='dt',freq='1min')).mean() 
    stt = []
    for ttime in df_slow['TAI']:
        tmp = ttime - leap_seconds
        ss = int(tmp)
        us = 1e6*(tmp-ss)
        stt.append(ta0 + dt.timedelta(seconds = ss, microseconds = us))
    sdate = np.asarray(stt)
    #stt = Time(df_slow['TAI'] - TaiOffset, format='unix_tai', scale='tai')
    #sdate = np.asarray(stt.datetime)
    
    s0 = np.asarray(df_slow['ESP_0_7_COUNTS'])
    s30 = np.asarray(df_slow['ESP_30_COUNTS'])
    del df_eve
    del df_slow
    
    e_data = [edate, e0, e30, sdate, s0, s30]
    
    return [error, e_data]
#    return edate, e0, e30, sdate, s0, s30
    
def get_latest_goes(debug=debug):
    error = [0]
    goes_fn_root = "https://services.swpc.noaa.gov/json/goes/"
    p6h = "primary/xrays-6-hour.json"
    s6h = "secondary/xrays-6-hour.json"
    
    try:
        df_tmp=pd.read_json(goes_fn_root+p6h)
    except:
        error = [-20]
#        return [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]
        return error
        
    long_p = df_tmp[df_tmp["energy"]=="0.1-0.8nm"]
    short_p = df_tmp[df_tmp["energy"]=="0.05-0.4nm"]    
    tmp = df_tmp["satellite"]
    gp = np.asarray(tmp)[-1]

    df_tmp=pd.read_json(goes_fn_root+s6h)
    long_s = df_tmp[df_tmp["energy"]=="0.1-0.8nm"]
    short_s = df_tmp[df_tmp["energy"]=="0.05-0.4nm"]
    tmp = df_tmp["satellite"]
    gs = np.asarray(tmp)[-1]

    times_p_tmp = np.asarray(long_p["time_tag"])
    long_p_tmp = np.asarray(long_p["observed_flux"])
    short_p_tmp =np.asarray(short_p["observed_flux"])

    times_s_tmp = np.asarray(long_s["time_tag"])
    long_s_tmp = np.asarray(long_s["observed_flux"])
    short_s_tmp =np.asarray(short_s["observed_flux"])


    gp_dat = []
    gp_long = []
    gp_short = []

    for ii in range(times_p_tmp.size):
        tmp2 = dt.datetime.strptime(times_p_tmp[ii], '%Y-%m-%dT%H:%M:%SZ')
        gp_dat.append(tmp2)
        gp_long.append(float(long_p_tmp[ii]))
        gp_short.append(float(short_p_tmp[ii]))

    gs_dat = []
    gs_long = []
    gs_short = []

    for ii in range(times_s_tmp.size):
        tmp2 = dt.datetime.strptime(times_s_tmp[ii], '%Y-%m-%dT%H:%M:%SZ')
        gs_dat.append(tmp2)
        gs_long.append(float(long_s_tmp[ii]))
        gs_short.append(float(short_s_tmp[ii]))
        
    gp_dat = np.asarray(gp_dat)
    gp_long = np.asarray(gp_long)
    gp_short = np.asarray(gp_short)
    
    gs_dat = np.asarray(gs_dat)
    gs_long = np.asarray(gs_long)
    gs_short = np.asarray(gs_short)
    
    g_data = [gp_dat, gp_long, gp_short, gs_dat, gs_long, gs_short, gp, gs]
    
    return [error, g_data]
    #return gp_dat, gp_long, gp_short, gs_dat, gs_long, gs_short, gp, gs
    

In [ ]:
eve_fn = "https://lasp.colorado.edu/eve/data_access/eve_data/quicklook/L0CS/LATEST_EVE_L0CS_DIODES_10s_counts.json"
df_eve = pd.read_json(eve_fn)

#df_eve.head()

# New Version Stuff (see EVE_FastFlareCheckRC0)

## Fit EVE to GOES

In [ ]:
#https://stackoverflow.com/questions/32723150/rounding-up-to-nearest-30-minutes-in-python
def find_end_time(tin, delta):
    return tin + (dt.datetime.min - tin) % delta

In [ ]:
# Fit GOES to EVE
mm_a = 7.288781677851528e-09; cc_a = -4.698588594426906e-06 # first guess each fit will improve, OR check for bad fit

In [ ]:
def do_it(debug=debug):
    error = [0]
#    e_dat, e0, e30, s_dat, s0, s30 
#    e_dat, e0, e30, s_dat, s0, s30 = get_latest_eve()

    e_list = get_latest_eve()
    error = e_list[0]
    if error[0] < 0:
        print('***')
        return [e_list[0]]
    
    
#    gp_dat, gp_long, gp_short, gs_dat, gs_long, gs_short, gp, gs  = get_latest_goes()
    g_list  = get_latest_goes()
    error = g_list[0]
    if error[0] < 0:
        return [g_list[0]]
    
# decompose goes data lists
    e_data = e_list[1]
    g_data = g_list[1]
    
    e_dat = e_data[0]; e0 = e_data[1]; e30 = e_data[2]
    s_dat = e_data[3]; s0 = e_data[4]; s30 = e_data[5]
    
    gp_dat = g_data[0]; gp_long = g_data[1]; gp_short = g_data[2]
    gs_dat = g_data[3]; gs_long = g_data[4]; gs_short = g_data[5]
    gp = g_data[6]; gs = g_data[7]
    
    

# Make common data grid
    grid_length = 2 # hrs
    grid_space = 30 # sec
# Need to find end time that can have interpolation for all channels
    latest = [np.max(e_dat),np.max(gp_dat), np.max(gs_dat) ]
    end_tmp = np.min(latest)
    end_tmp1 = end_tmp - dt.timedelta(seconds=grid_space)
    end_time = find_end_time(end_tmp1, dt.timedelta(seconds=grid_space))
    start_time = end_time - dt.timedelta(hours = grid_length)
    
# Find start time of GOES data
    earliest = [np.min(gp_dat), np.min(gs_dat)]
    g_tmp = np.max(earliest)
    g_tmp1 = g_tmp + dt.timedelta(seconds=grid_space)
    g_time = find_end_time(g_tmp1, dt.timedelta(seconds=grid_space))



    if debug:
        print(start_time)
        print(end_time)
        print(end_tmp)

    time_grid = np.arange(start_time, end_time,dt.timedelta(seconds=grid_space) )
    time_grid_all = np.arange(g_time, end_time,dt.timedelta(seconds=grid_space) ) 

#    if debug:
#        for tt in time_grid:
#            print(tt)


# interpolations
#https://stackoverflow.com/questions/54304423/why-does-a-conversion-from-np-datetime64-to-float-and-back-lead-to-a-time-differ
    xx= time_grid.astype("float")
    xx_all = time_grid_all.astype("float")

# convert DTs
# GOES Pri
    xxtmp = []
    for tt in gp_dat:
        xxtmp.append(np.datetime64(tt))
    xxtmp = np.asarray(xxtmp)
    xgp = xxtmp.astype("float")
    gpl_i = np.interp(xx,xgp,gp_long )
    gps_i = np.interp(xx,xgp,gp_short )
    
    gpl_ia = np.interp(xx_all,xgp,gp_long )
    gps_ia = np.interp(xx_all,xgp,gp_short )


# GOES Sec
    xxtmp = []
    for tt in gs_dat:
        xxtmp.append(np.datetime64(tt))
    xxtmp = np.asarray(xxtmp)
    xgs = xxtmp.astype("float")
    gsl_i = np.interp(xx,xgs,gs_long )
    gss_i = np.interp(xx,xgs,gs_short )
    
    gsl_ia = np.interp(xx_all,xgs,gs_long )
    gss_ia = np.interp(xx_all,xgs,gs_short )
    

#EVE fast
    xxtmp = []
    for tt in e_dat:
        xxtmp.append(np.datetime64(tt))
    xxtmp = np.asarray(xxtmp)
    xe = xxtmp.astype("float")
    e0_i = np.interp(xx,xe,e0 )


#EVE slow
    xxtmp = []
    for tt in s_dat:
        xxtmp.append(np.datetime64(tt))
    xxtmp = np.asarray(xxtmp)
    xs = xxtmp.astype("float")
    s0_i = np.interp(xx,xs,s0 )
    
    
    gl = np.maximum(gpl_i, gsl_i)
    gl_all =  np.maximum(gpl_ia, gsl_ia)
    gs = np.maximum(gps_ia, gss_ia)
    gr = gs/(gl_all+1e-15)
    
    if debug:
        plt.plot(time_grid, gpl_i/np.max(gpl_i), 'r')
        plt.plot(time_grid, gsl_i/np.max(gsl_i), 'b')
        plt.plot(time_grid, e0_i/np.max(e0_i), 'g')
        plt.plot(time_grid, s0_i/np.max(s0_i), '.')
        plt.show()
        
# Make list to return data
    g_raw_list = [gp_dat, gp_long, gp_short, gs_dat, gs_long, gs_short]
    g_intp_list = [time_grid,time_grid_all, gl,gs, gr, gpl_i, gsl_i]
    eve_raw_list = [e_dat, e0, e30, s_dat, s0, s30]
    eve_intp_list = [e0_i, s0_i]
    
    r_list = [error, g_raw_list, g_intp_list, eve_raw_list, eve_intp_list ]

    return r_list

In [ ]:
def do_slope_pred(e_dat,s_dat, time_range, e0, s0):
    ct = dt.datetime.utcnow()
    st = ct - dt.timedelta(hours = float(time_range))
    time_good = np.where(e_dat >= st)
#    good = np.where((e0 > 100) & (e30 <2000))
    te0 = e0[time_good]
    xx = e_dat[time_good]
    ft = []

    npts = len(te0)
    grad0=[]
    if npts > 1: # do the gradients stuff
        for jj in range(npts):
            ft.append(xx[jj])
            if jj==0:
                grad0.append(0)
            else:
                grad0.append(e0[jj] - e0[jj-1])
    grad0 = np.asarray(grad0)
    ft = np.array(ft)
    
    grad_s=[]
    
    #slow gradient slope
    npts_s = len(s0)
    st = []
    if npts_s > 1: # do the gradients stuff
        for jj in range(npts_s):
            st.append(s_dat[jj])
            if jj==0:
                grad_s.append(0)
            else:
                grad_s.append(s0[jj] - s0[jj-1])
    grad_s = np.asarray(grad_s)
    st=np.asarray(st)
    ns_pred = -5
    tt_s = np.arange(-1*ns_pred)
    yy_s = grad_s[ns_pred:]
    slope_s, intercept_s = np.polyfit(tt_s, yy_s, deg=1)

  
    # grad now for prediction
    n_pred = -25
    tt = np.arange(-1*n_pred)
    yy = e0[n_pred:]
    slope, intercept = np.polyfit(tt, yy, deg=1)
    # forward 
    xxpred = np.arange(-1*n_pred,-5*n_pred)
    yypred = []
    ttpred = []
    slope_av = 0.6*(0.018*slope_s + 0.982*slope)
    for ii in range(len(xxpred)):
        ttpred.append(xx[-1] + dt.timedelta(seconds = 10*ii))
        yypred.append(e0[-1] + ii*slope_av)
    ttpred=np.asarray(ttpred)
    yypred=np.asarray(yypred)
    error = [0]
    pred_array = [ttpred, yypred]
    grad_array = [ft, grad0, st, grad_s]
    return [error, pred_array, grad_array]
 

In [ ]:
m_hist = []
c_hist = []
g_hist = [] # Goes L flux

In [ ]:
# To Do
# add latency times
# scaling of esp plots to last 30 mins ?
# consistent gradient calculations
# organize into defines
# Fast slope plot seems wrong
# Add exception handling to do_it call
# Set initial fit parameters based on measured DN so it gets the solution faster


# From Tom E.'s email 10/3/2023 11:59am
r2t_coef =  [2.74603592e+00, 1.29468042e+02, -9.66284279e+02, 5.51752562e+03, \
             -1.86637781e+04, 3.59506086e+04, -3.60993534e+04, 1.46869290e+04]

time_range = 3
loop_update_time = 20 #seconds to wait before looking for data again
try:
    while True:
        #time_grid, time_grid_all, gl, gs, gr, gpl_i, gsl_i, e0_i, s0_i,e_dat, e0  = do_it(debug = False)
        rl = do_it(debug = False)    #<<< Add try/except
        # decompose return list
        error = rl[0]
        if error[0] == 0:
            g_raw_list = rl[1];  g_intp_list= rl[2]; eve_raw_list= rl[3]; eve_intp_list = rl[4]
        
            time_grid = g_intp_list[0];  time_grid_all= g_intp_list[1]
            gl= g_intp_list[2];  gs= g_intp_list[3]; gr= g_intp_list[4]
            gpl_i = g_intp_list[5];  gsl_i = g_intp_list[6]
            e0_i = eve_intp_list[0]; s0_i = eve_intp_list[1]
            e_dat = eve_raw_list[0]; e0 = eve_raw_list[1] ; e30 = eve_raw_list[2] 
            s_dat = eve_raw_list[3]; s0 = eve_raw_list[4] ; s30 = eve_raw_list[5]
            gp_dat = g_raw_list[0]
        
        
        # Do Plots
#        if (error == 0): # make sure we were able to get data

            mm, cc = np.polyfit(s0_i, gl, deg=1 )
# Time stuff
            ct = dt.datetime.utcnow()
            pt= ct + dt.timedelta(hours = float(0.25))
            st = ct - dt.timedelta(hours = float(time_range))    

    #### OLD stuff #######
            # extract just the data we want
            
            time_good = np.where(e_dat >= st)
#    good = np.where((e0 > 100) & (e30 <2000))
            te0 = e0[time_good]
            te30 = e30[time_good]
    
    
    
# Scaling
            c30_max = np.max(te30)# 300
            c30_min = np.min(te30)# 190
            c30_scl = c30_max - c30_min
     
            c0_max = np.max(te0)
            c0_min = np.min(te0)
            c0_scl = c0_max - c0_min
    
    



            pe0 = (te0 - c0_min)/c0_scl
            ps0 = (s0 - c0_min)/c0_scl
#           pe30 = (e30[good]- 1.0*np.min(e30[good]))/np.max(e30[good])
            pe30 = (te30- c30_min)/c30_scl
            ps30 = (s30- c30_min)/c30_scl
        
            xx =  e_dat[time_good]
            nptso = len(te0)
            grad0o=[]
            if nptso > 1: # do the gradients stuff
                for jj in range(nptso):
                    if jj==0:
                        grad0o.append(0)
                    else:
                        grad0o.append(e0[jj] - e0[jj-1])
            grad0o = np.asarray(grad0o)
    
            grad_so=[]
    
    #slow gradient slope
            npts_so = len(s0)
            if npts_so > 1: # do the gradients stuff
                for jj in range(npts_so):
                    if jj==0:
                        grad_so.append(0)
                    else:
                        grad_so.append(s0[jj] - s0[jj-1])
            grad_so = np.asarray(grad_so)
            ns_predo = -5
            tt_s = np.arange(-1*ns_predo)
            yy_s = grad_so[ns_predo:]
            slope_s, intercept_s = np.polyfit(tt_s, yy_s, deg=1)

    
            # grad now for prediction
            n_predo = -25
            tt = np.arange(-1*n_predo)
            yy = pe0[n_predo:]
            slope, intercept = np.polyfit(tt, yy, deg=1)
            # forward 
            xxpred = np.arange(-1*n_predo,-5*n_predo)
            yypred = []
            ttpred = []
            slope_av = 0.6*(0.018*slope_s + 0.982*slope)
            for ii in range(len(xxpred)):
                ttpred.append(xx[-1] + dt.timedelta(seconds = 10*ii))
                yypred.append(ps0[-1] + ii*slope_av)


# Update fit coefficients, need to think of the best way to do this i.e. limits to update, start again, or keep existing
            if ((np.abs((mm_a - mm)/mm_a) <0.99) and (np.abs((cc_a - cc)/cc_a) < 0.99)):
                # print("updating m, c")
                mm_a = mm_a/2 + mm/2
                cc_a = cc_a/2 +cc/2
            else:
                mm_a = mm_a
                cc_a = cc_a
            fig_test = plt.figure(figsize=(16,8))
            #gs = gridspec.GridSpec(4, 1, height_ratios=[3, 1, 1, 1]) 
        
            # Time stuff
            #ct = dt.datetime.utcnow()
            #pt= ct + dt.timedelta(hours = float(0.25))
            #st = ct - dt.timedelta(hours = float(time_range))
            
            #    error = [0]
            #    pred_array = [ttpred, yypred]
            #    grad_array = [ft, grad0, st, grad_s]
            #    return [error, pred_array, grad_array]
        
            #tp, yp, ft, fs, slt, ss = do_slope_pred(e_dat, s_dat, time_range, e0, s0)
            rl = do_slope_pred(e_dat, s_dat, time_range, e0, s0)
            error = rl[0]
            if error[0] == 0: # need to find a better place for this, but this error is defined as 0 at the moment
                pred_array = rl[1]; grad_array = rl[2]
                tp = pred_array[0]
                yp = pred_array[1]
                ft = grad_array[0]
                fs = grad_array[1]
                slt = grad_array[2]
                ss = grad_array[3]
                
        
            g_t = r2t_coef[7] * gr**7 + r2t_coef[6] * gr**6 + r2t_coef[5] * gr**5 + \
              r2t_coef[4] * gr**4  +r2t_coef[3] * gr**3 + r2t_coef[2] * gr**2 + \
              r2t_coef[1]* gr + r2t_coef[0]
        

            fig_test.add_subplot(4,1,1) # <<<<<<<<<<<<<<<<<<<<<<<<<< XRS EVE Plot
            #plt.axhline(y= 1e-5,  color='red')
            #plt.axhline(y= 1e-6,  color='yellow')


            plt.plot(time_grid, gl, color='red', label = 'GOES Long (fit length)')
            plt.plot(time_grid_all, gs, color='blue', label = 'GOES Short')
        
            plt.plot(e_dat, mm_a*e0 + cc_a, color='orange', linestyle = ' ', marker = '+', markersize = 0.5, label = 'EVE 0 fit')
            plt.plot(tp, mm_a*yp + cc_a, 'g:', label = 'EVE Predict')
            
            plt.yscale('log')
            plt.ylabel('GOES XRS (W/m2)')

            plt.ylim(1e-9, 1e-4)
            plt.axhline(y=1e-5, linestyle = '--',color='green')
            plt.xlim(st,pt)
            plt.xticks(color='w')
            plt.axvline(x= ct, linestyle = '--', color='black')
            tnow = dt.datetime.strftime(ct, "%H:%M:%S")
            plt.text(ct, 1.3e-4, tnow, horizontalalignment='center')
            #plt.title('------ Combined XRS - EVE ------')
            plt.grid()
            plt.legend(loc = 'lower left')
            
            fig_test.add_subplot(4,1,2) # <<<<<<<<<<<<<<<<<<<<<<<<<< Plot I like
            # set scaling time for normalization
            t_scale = 60 # minutes
            
            #fast
            plt.plot(xx,pe0, color='cyan')
            plt.plot(ttpred, yypred, 'b:', label = 'Simple Predict') # <<<< prediction part
            plt.plot(xx,(pe30 - np.min(pe30)), color='orange')    
            #slow
            plt.plot(s_dat, ps0, 'b', label = 'ESP 0-7 nm')
            plt.plot(s_dat, ps30 - np.min(pe30), 'r', label = 'ESP 30 nm')
    



            plt.ylim(-0.25,1.25)
    
    #plt.xlabel('Time (UTC)')
            #tnow = dt.datetime.strftime(ct, "%H:%M:%S")
            #plt.text(ct, 1.3, tnow, horizontalalignment='center')
    
    
            plt.xticks(color='w')
            plt.ylabel('Normalized Counts')
            plt.xlim(st,pt)
            plt.axvline(x= ct, linestyle = '--', color='black') 
            plt.legend(loc = 'upper left')
            plt.grid()

        
            fig_test.add_subplot(4,1,3) # <<<<<<<<<<<<<<<<<<<<<<<<<< XRS Temperature

            plt.plot(time_grid_all,g_t, 'g-')
            plt.xlim(st,pt)
            plt.axvline(x= ct, linestyle = '--', color='black')
            plt.ylabel('T (MK)')
            plt.xticks(color='w')
            
            xmin, xmax, ymin, ymax = plt.axis()
            td = int(time_range *20)
            #plt.text(ct - dt.timedelta(minutes=td), 0.9*ymax, delay_g, color='k', fontsize = 12, bbox=dict(facecolor='white'))
            plt.grid()
 
            #plt.plot(e_dat, e30, color='orange', label = '30 nm 10 s' )
            #plt.plot(s_dat, s30, color='red', label = '30 nm 1 min' )
            #plt.xlim(st,pt)
            #plt.axvline(x= ct, linestyle = '--', color='black')
            #plt.grid()
            #plt.legend(loc = 'lower left')
            ##plt.xlabel('Time (UTC)')
            #plt.xticks(color='w')
            #plt.ylabel('ESP 30 nm CPS')
            
            fig_test.add_subplot(4,1,4) # <<<<<<<<<<<<<<<<<<<<<<<<<< ESP 30 nm 
            plt.plot(ft, fs, color='grey', label = 'Fast Slope' )# < this is delayed I think
            plt.plot(slt, ss, color='black', label = 'Slow Slope' )
            plt.xlim(st,pt)
            plt.axvline(x= ct, linestyle = '--', color='black')
            plt.grid()
            plt.legend(loc = 'lower left')
            plt.xlabel('Time (UTC)')
            plt.ylabel('Delta CPS')
        
        
            plt.show()


            m_hist.append(mm_a)
            c_hist.append(cc_a)
            # print(mm_a, cc_a)
            #time.sleep(60)
            
    
# Wait loop so we don't spam the servers
            print('Waiting: ', end=' ')
            for jj in range(int(loop_update_time/5)):
                tn = dt.datetime.utcnow()
                print(loop_update_time - jj*5, end='s ')
                time.sleep(5)
            print(" Fetching New Data ", end=' ')
            clear_output(wait=True)
    
except KeyboardInterrupt: 
    print('interrupted!')

In [ ]:
# Need to stop data plots to do this step

m_ave = sum(m_hist) / len(m_hist)
c_ave = sum(c_hist) / len(c_hist)

fig_fit_history = plt.figure(figsize=(16,4))

fig_fit_history.add_subplot(1,2,1)
plt.plot(m_hist)
plt.axhline(y= m_ave)
plt.title = ("slope")

fig_fit_history.add_subplot(1,2,2)
plt.plot(c_hist)
plt.axhline(y= c_ave)
plt.title = ("offset")

plt.show()

print('Slope  avereage: {}'.format(m_ave))
print('Offset avereage: {}'.format(c_ave))

In [ ]:
tl = get_latest_eve(debug=True)

In [ ]:
error = tl[0]
if error[0] < 0:
    print(error[0])

In [ ]:
print(rl[0])

# Old

In [ ]:
tai_max = (np.max(df_eve['TAI']))
#TaiOffset = 378691200 + 37
#TaiOffset = 37
leap_seconds = 37
    
# tt = Time(tai_max - TaiOffset, format='unix_tai', scale='tai')
# Don't use astropy for Juno
ta0 = dt.datetime(1958,1,1,0,0,0,0) # TAI base time
tt = []
for ttime in df_eve['TAI']:
    tmp = ttime - leap_seconds
    ss = int(tmp)
    us = 1e6*(tmp-ss)
    tt.append(ta0 + dt.timedelta(seconds = ss, microseconds = us))
tt = np.asarray(tt)

print(tai_max, tt, tt.datetime)

tmp = (tai_max - TaiOffset)
print(tmp)

ss = int(tmp)
us = 1e6*(tmp-ss)
print(ss, us)

dt0 = datetime.datetime(1958,1,1,0,0,0,0)
print(dt0)

dt1 = dt0 + datetime.timedelta(seconds = ss, microseconds = us)
print(dt1)

leap_seconds = 37
dt0 = datetime.datetime(1958,1,1,0,0,0,0) # TAI base time
tt = []
for ttime in df_eve['TAI']:
    tmp = ttime - leap_seconds
    ss = int(tmp)
    us = 1e6*(tmp-ss)
    tt.append(dt0 + datetime.timedelta(seconds = ss, microseconds = us))
eve_time = np.asarray(tt)


e0 =  df_eve['ESP_0_7_COUNTS'].to_numpy()

if True:
    npts = len(e0)
    grad0=[]
    if npts > 1: # do the gradients stuff
        for jj in range(npts):
            if jj==0:
                grad0.append(0)
            else:
                grad0.append(e0[jj] - e0[jj-1])
    grad0 = np.asarray(grad0)


plt.plot(eve_time, e0)
plt.plot(eve_time, 50*grad0)
plt.show()